In [444]:
import pandas as pd
import numpy as np
import re
from langdetect import detect
from sklearn.model_selection import train_test_split
import cld3
import math
from baseline import *
from evaluation import *

ModuleNotFoundError: No module named 'rouge'

In [449]:
train_subset = train.iloc[:1000]

In [450]:
train_subset.to_csv('data/train_subset.csv', index=False)

In [448]:
train = pd.read_csv("data/csv/train.csv")
train.head()

,Artist,Song,Genre,Language,Lyrics,reliable_lang
0,belle sebastian,legal man,Pop,en,"L-O-V-E love, it's coming back, it's coming ba...",en
1,pete townshend,a friend is a friend,Rock,en,When eyes meet in silence\nA pact can be made\...,en
2,blossoms,love talk,Rock,en,my eyes align with you\nyou're on my side\nand...,en
3,bukka white,parchman farm blues,Folk,en,Judge gimme me life this morn'in\nDown on Parc...,en
4,pat benatar,i won't,Rock,en,I was there when you cried like a baby\nWhen y...,en


In [315]:
data = pd.read_csv("data/train.csv")
data = data[data['Language']=='en']
data = data[data['Lyrics'].apply(lambda x: '\n' in x)]
data = data.drop_duplicates(subset=['Artist', 'Song', 'Genre'], ignore_index=True)
data = data[data['Lyrics'].apply(lambda x: '---------' not in x)]
data['Lyrics'] = data['Lyrics'].apply(lambda x: re.sub('\[.*\]', ' ', x))

In [316]:
def check_en(line):
    result = cld3.get_language(line)
    lang = result[0]
    reliable = result[2]
    if reliable and lang == 'en':
        return 'en'
    else:
        return 'other'

In [328]:
len(data)

192123

In [317]:
data['reliable_lang'] = data['Lyrics'].apply(check_en)
data = data[data['reliable_lang']=='en']

In [321]:
indices = np.arange(len(data))
train_index, test_index = train_test_split(indices, test_size = 0.1)
train_index, dev_index = train_test_split(train_index, test_size=0.1)

train_data = data.iloc[train_index].reset_index(drop=True)
dev_data = data.iloc[dev_index].reset_index(drop=True)
test_data = data.iloc[test_index].reset_index(drop=True)

In [322]:
train_data.to_csv('data/csv/train.csv', index=False)
dev_data.to_csv('data/csv/dev.csv', index=False)
test_data.to_csv('data/csv/test.csv', index=False)

In [294]:
data['Genre'].value_counts()

Rock          96376
Pop           70438
Metal         15910
Jazz          11701
Folk           6922
Indie          5592
R&B            1572
Electronic      447
Name: Genre, dtype: int64

In [288]:
df = pd.read_csv('data/csv/test.csv')

In [290]:
df = df[df['Genre']=='Pop']

In [280]:
dev_data['Genre'].value_counts()

Rock          8679
Pop           6313
Metal         1490
Jazz          1035
Folk           603
Indie          497
R&B            171
Electronic      36
Name: Genre, dtype: int64

In [184]:
jazz = data[data['Genre']=='Jazz'].reset_index(drop=True)
indices = np.arange(len(jazz))
train_index, test_index = train_test_split(indices, test_size = 0.1)
train_index, dev_index = train_test_split(train_index, test_size=0.1)

In [242]:
path = 'data/train/jazz_train.txt'
with open(path,'w', encoding='utf-8', errors='ignore') as f:
    for i in train_index:
#         f.write("ARTIST NAME: "+jazz['Artist'].iloc[i]+'\n')
#         f.write("GENRE: "+jazz['Genre'].iloc[i]+'\n')
        f.write(jazz['Lyrics'].iloc[i])
        f.write('\n')

In [116]:
for g in list(data['Genre'].value_counts().index):
    df = data[data['Genre']==g]
    artist = np.array(df['Artist']).reshape(-1,1)
    genre = np.array(df['Genre']).reshape(-1,1)
    lyrics = np.array(df['Lyrics']).reshape(-1,1)

    out = np.hstack((artist, genre, lyrics))
    file_path = 'data/'+g+'_train.npy'
    np.save(file_path, out)

In [105]:
artist = np.array(rnb['Artist']).reshape(-1,1)
genre = np.array(rnb['Genre']).reshape(-1,1)
lyrics = np.array(rnb['Lyrics']).reshape(-1,1)

rnb_out = np.hstack((artist, genre, lyrics))
np.save('data/rnb_train.npy', rnb_out)

In [93]:
detect(rnb['Lyrics'].iloc[0])

'en'

In [90]:
rnb['detect_lang'] = rnb['Lyrics'].apply(lambda x: detect(x[:50]))

/Users/haowei/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [80]:
data['Genre'].value_counts()

Rock          103721
Pop            83620
Metal          17207
Jazz           11760
Folk            7023
Indie           6033
R&B             1575
Electronic       462
Name: Genre, dtype: int64

In [39]:
print(rnb.iloc[20]['Lyrics'])


 

so many people 
I know only a few 
yes I may say that I love this man 
and that man 
but what keeps me from loving you? 
date of birth  geography 
the color of my skin  ideology 
you got ten fingers  two legs  one nose 
like me 
just like me 
and it's as simple as that 
you see 
and if I don't know who to love 
I love them all 
and if I don't know who to trust 
I trust them all 
and if I don't know who to kill 
I may kill myself instead 
from the mouth of a baby 
will come the world-saving words 
that will save us all 
and from the lungs of a child 
will come the everlasting breath of god 
increasing peace and honesty 
and not carrying on despite of me 
don't you know 
this ain't about no race, no creed 
no race, no creed 
and it's as simple as that 
you see 
and if I don't know who to love 
I love them all 
and if I don't know who to trust 
I trust them all 
and if I don't know who to kill 
no suicide 
I'm already dead 




In [398]:
train_df['Genre'].value_counts()

Rock          70156
Pop           53050
Metal         11728
Jazz           9457
Folk           5143
Indie          4475
R&B            1258
Electronic      352
Name: Genre, dtype: int64

In [441]:
def create_ngram_model(model_class, path, n=2, k=0, type='csv', genre=None):
    ''' Creates and returns a new n-gram model trained on path file '''
    """
    path: numpy file of dimension nx3, columns: artist, genre, lyrics
    
    return: updated model of model_class
    """
    num_iterations = 100  # number of samples to draw
    length = 20  # how many lines in one sample

    model = model_class(n, k)
#     if type == 'txt':
#         with open(path, encoding='utf-8', errors='ignore') as f:
#             lyrics = f.read()
#     else:
#         lyrics = csv_to_text(path, genre)

#     for i in range(num_iterations):
#         temp = lyrics_chunks(lyrics, length)
#         model.update(temp)
    df = pd.read_csv(path)
    df = df[df['Genre']==genre]
    for i in range(len(df)):
        lyric = df.iloc[i]['Lyrics']
        model.update(lyric)

    return model

def start_pad(n):
    ''' Returns a padding string of length n to append to the front of text
        as a pre-processing step to building n-grams '''
    return '~' * n

def ngrams(n, text):
    ''' Returns the ngrams of the text as tuples where the first element is
        the length-n context and the second is the character '''
    adjusted_txt = start_pad(n) + text
    gramlist = []
    for i in range(n, len(adjusted_txt)):
        ngram = (adjusted_txt[i-n:i], adjusted_txt[i])
        gramlist.append(ngram)
    return gramlist

def csv_to_text(path, genre=None):
    ''' Takes in a csv file and return the lyrics concatenated into one text file'''
    df = pd.read_csv(path)
    if genre:
        df = df[df['Genre']==genre]
    lyrics = '\n'.join(list(df['Lyrics']))
    return lyrics

def lyrics_chunks(lyrics, length):
    """
    randomly samples lyrics chunks of specified length

    :param lyrics: str, entire lyrics corpus
    :param length: int, number of lines in one lyrics chunks
    :return: str, lyric lines joined by new line
    """
    lyrics_lines = lyrics.split('\n')
    start_idx = random.randint(0, len(lyrics_lines)-1)
    end_idx = min(start_idx+length, len(lyrics_lines))
    selected_lyrics = lyrics_lines[start_idx:end_idx]
    return '\n'.join(selected_lyrics)

################################################################################
# Model
################################################################################

class NgramModel(object):
    ''' A basic n-gram model using add-k smoothing '''

    def __init__(self, n, k):
        self.n = n
        self.k = k
        self.vocabs = []
        self.context_list = {}
        self.context_count = {}

    def get_vocab(self):
        ''' Returns the set of characters in the vocab '''
        return set(''.join(self.vocabs))

    def update(self, text):
        ''' Updates the model n-grams based on text '''
        self.vocabs.append(text)
        # self.sorted_vocabs = list(set(self.vocabs))
        # self.sorted_vocabs.sort()
        gramlist = ngrams(self.n, text)
        for pair in gramlist:
            context = pair[0]
            char = pair[1]
            if context in self.context_list: # context previously showed up
                self.context_count[context]+=1
                if char in self.context_list[context]: # combination previously showed up
                    self.context_list[context][char]+=1
                else:
                    self.context_list[context][char]=1
            else:
                self.context_count[context]=1
                self.context_list[context] = {char: 1}

    def prob(self, context, char, lambdas=None):
        ''' Returns the probability of char appearing after context '''
        if context not in self.context_list:
            return 1/len(self.get_vocab())

        # if char not in self.context_list[context]:
        #     return 0.0
        # else:
        #     return self.context_list[context][char]/self.context_count[context]
        count = 0
        if char in self.context_list[context]:
            count = self.context_list[context][char]
        p = (count+self.k)/(self.context_count[context]+self.k*len(self.get_vocab()))
        return p

    def random_char(self, context):
        ''' Returns a random character based on the given context and the
            n-grams learned by this model '''
        vocabs = list(self.get_vocab())
        vocabs.sort()
        total_prob = 0
        r = random.random()
        for i in range(len(vocabs)):
            total_prob += self.prob(context, vocabs[i])
            if total_prob > r:
                return vocabs[i]
        return vocabs[-1]

    def random_text(self, length, context=None):
        ''' Returns text of the specified character length based on the
            n-grams learned by this model '''
        if context == None:
            context = start_pad(self.n)
        else:
            context = context[-self.n:]
        text = ''
        for i in range(length):
            char = self.random_char(context)
            text += char
            context = context[1:]+char
        return text

    def random_lines(self, num_lines, context=None):
        ''' Returns specified number of lines based on learned n-grams model'''
        if context == None:
            context = start_pad(self.n)
        else:
            context = context[-self.n:]
        text = ''
        count = 0
        while True:
            char = self.random_char(context)
            if char == '\n':
                count += 1
                if count == num_lines:
                    break
            text += char
            context = context[1:] + char
        return text
    
    def perplexity(self, text,lambdas=None):
        ''' Returns the perplexity of text based on the n-grams learned by
            this model '''
        padded_text = start_pad(self.n) + text
        context = start_pad(self.n)
        log_pp = 0
        for i in range(self.n, len(padded_text)):
            p = self.prob(context, padded_text[i])
            if p == 0:
                return float('inf')
            log_pp += math.log(p)
            context = context[1:]+padded_text[i]
        log_pp = log_pp/(-len(text))
        pp = math.exp(log_pp)
        return pp
    
    def gen_next_sent(self, input_sent):
        ''' Returns the next sentence prediction based on input sentence '''
        input_sent = start_pad(self.n) + input_sent + '\n'
        context = input_sent[-self.n:]
        next_sent = ''
        while True:
            char = self.random_char(context)
            if char != '\n':
                next_sent += char
                context = context[1:] + char
            else:
                break
        return next_sent

In [442]:
rnb = create_ngram_model(NgramModel, 'data/csv/train.csv', 4,0.0000001,genre='R&B')

In [437]:
rnb6 = create_ngram_model(NgramModel, 'data/csv/train.csv', 6,0.0000001,genre='R&B')

In [443]:
rnb.gen_next_sent('I like')

'I will in my loaded came'

In [432]:
indie = create_ngram_model(NgramModel, 'data/csv/train.csv', 5,0.0000001,genre='Indie')
print(indie.random_lines(10))

Sweet apology
Then bad line of these world whiskey and sleep 'cause you're nothings that's wrong (shout
You thing's the wet myself out for You

My little more electric with dreamed Sadie
Yeah, you weren't wear I've that bring is up
Rider of us don't be a funk that family.
The airport to hide you
(Verse)
I wrote and stuffed in the way I hear


In [433]:
indie6 = create_ngram_model(NgramModel, 'data/csv/train.csv', 6,0.0000001,genre='Indie')
print(indie6.random_lines(10))

Greasy here 
What a Swedish generation?

Am I repeated)
I luv ya (layered little now
Oh, you must be having now 
I'll take my past if for the ooooooooooo
there's a lesson to the need to take
we're both breadcrumbs
And you could ask what's all you


In [427]:
def avg_perplexity(model, test_df, model_type='ngram', genre=None):
    """
    computes perplexity of unseen text using given model

    :param model: learned model
    :param test_df:
    :param model_type: 'ngram' or 'neural', default is 'ngram'
    :return: average perplexity score from 20 songs
    """
    tot_pp = 0
    num_samp = 150
    
    if genre:
        test_df = test_df[test_df['Genre']==genre]
    
    if model_type == 'ngram':
        # select 20 random songs from test data and compute perplexity
        for i in range(num_samp):
            idx = random.randint(0,len(test_df)-1)
            lyric = test_df.iloc[idx]['Lyrics']
            tot_pp += model.perplexity(lyric)

#     if model_type == 'neural':
#         for i in range(num_samp):
#             idx = random.randint(len(test_df))
#             lyric = test_df.iloc[idx]['Lyrics']
#             tot_pp += nn_perplexity(lyric, model)

    return tot_pp / num_samp

# def nn_perplexity(val_text, model):
#     input = char_tensor(val_text[:-1])
#     target = char_tensor(val_text[1:])
#     hidden = model.init_hidden()
#     criterion = nn.CrossEntropyLoss()
#     loss = 0
#     for i in range(len(val_text)-1):
#         output, hidden = model(input[i], hidden)
#         loss += criterion(output, target[i].view(-1))
#     avg_loss = loss.item()/(len(val_text)-1)
#     return np.exp(avg_loss)

# def char_tensor(string):
#     tensor = torch.zeros(len(string)).long()
#     for c in range(len(string)):
#         tensor[c] = all_characters.index(string[c])
#     return Variable(tensor)

In [396]:
test_df = pd.read_csv('data/csv/test.csv')
train_df = pd.read_csv('data/csv/train.csv')

In [351]:
idx = random.randint(0,len(test_df)-1)
lyric = test_df.iloc[idx]['Lyrics']
# tot_pp += model.perplexity(lyric)

In [430]:
print(avg_perplexity(rnb, test_df,'ngram', 'R&B'))
print(avg_perplexity(rnb, test_df,'ngram', 'Indie'))

KeyboardInterrupt: 

In [167]:
model = NgramModel(5,0)
for i in range(5000):
    model.update(lyrics[i])

In [169]:
model.context_list.keys()

dict_keys(['~~~~~'])

In [170]:
print(model.random_text(100))

aIxPtb[KYIaP
F.dAFzEkN[FSTVDlnnoE,IfqlshVHmz]Js-pbPCWDPxcK-SYyIWL1JqUyrEyd
H1DOWS1c'mf:s
x'jygrbvrez
